# MTA Subway Ridership Analysis for Office Apocalypse Algorithm

## Overview
This notebook analyzes NYC MTA subway ridership data to understand **economic activity patterns** and **commuter accessibility** around office buildings. MTA ridership serves as a crucial indicator of neighborhood vitality and office district health that no other dataset can provide.

## Why MTA Data is Critical:
- **Economic Activity Proxy**: Ridership reflects neighborhood business activity and employment levels
- **Accessibility Metric**: Transit access directly impacts office building desirability and occupancy
- **Temporal Patterns**: Hourly/daily patterns reveal commuter vs. residential areas
- **Pre/Post-Pandemic Insights**: 2020-2024 data captures the office market transformation

## Unique Value Proposition:
- **Real-time Economic Health**: Ridership drops indicate declining business districts
- **Spatial Accessibility**: Distance to high-ridership stations affects office building value
- **Market Dynamics**: Commuter patterns reveal which areas recovered post-pandemic

## Dataset Source
- **File**: `MTA_Subway_Hourly_Ridership__2020-2024.csv`
- **Source**: Metropolitan Transportation Authority
- **Coverage**: All NYC subway stations, hourly ridership 2020-2024

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configure plotting
plt.style.use('default')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load and Explore MTA Dataset Structure

Let's examine the MTA ridership data to understand how it captures economic activity and commuter patterns that are essential for office building analysis.

In [ ]:
# Load MTA ridership dataset
mta_path = r"c:\Users\pcric\Desktop\capstone_project\office_apocalypse_algorithm_project\data\raw\MTA_Subway_Hourly_Ridership__2020-2024.csv"
print("Loading MTA Subway Ridership dataset...")
df_mta = pd.read_csv(mta_path)

print(f"Dataset shape: {df_mta.shape}")
print(f"Number of ridership records: {len(df_mta):,}")
print(f"Number of features: {len(df_mta.columns)}")
print("\n" + "="*60)
print("MTA Dataset Overview")
print("="*60)

In [ ]:
# Explore basic structure and identify key columns
print("Dataset Info:")
df_mta.info()
print("\n" + "="*50)
print("First 5 rows:")
print(df_mta.head())

print("\n" + "="*50)
print("Key Economic Indicators Available:")
print("="*50)

# Identify critical columns for economic analysis
critical_economic_columns = [
    'Station Name',     # Station identification
    'Borough',          # Geographic location
    'Date',            # Temporal analysis
    'Hour',            # Commuter pattern analysis
    'Entries',         # Ridership volume
    'Exits'            # Ridership volume
]

for col in df_mta.columns:
    print(f"✓ {col}")
    
print(f"\nColumns that capture economic activity:")
for col in critical_economic_columns:
    if any(keyword in col.lower() for keyword in ['station', 'borough', 'date', 'hour', 'entries', 'exits']):
        print(f"  → {col}: Essential for economic activity analysis")

## 2. MTA Economic Activity Visualizations

These visualizations demonstrate how MTA ridership data reveals economic patterns that no other dataset can capture.

In [ ]:
# MTA Economic Activity Pattern Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('MTA Ridership: Economic Activity Intelligence for Office Market Analysis', fontsize=16, fontweight='bold')

# Parse date and hour for temporal analysis
if 'transit_timestamp' in df_mta.columns or any('date' in col.lower() for col in df_mta.columns):
    # Find date/time columns
    date_cols = [col for col in df_mta.columns if any(keyword in col.lower() for keyword in ['date', 'time', 'timestamp'])]
    hour_cols = [col for col in df_mta.columns if 'hour' in col.lower()]
    
    print(f"Date/Time columns found: {date_cols}")
    print(f"Hour columns found: {hour_cols}")

# Station ridership analysis
if any('station' in col.lower() for col in df_mta.columns):
    station_cols = [col for col in df_mta.columns if 'station' in col.lower()]
    station_col = station_cols[0] if station_cols else None
    
    if station_col:
        # Top stations by ridership volume
        ridership_cols = [col for col in df_mta.columns if any(keyword in col.lower() for keyword in ['entries', 'exits', 'ridership'])]
        
        if ridership_cols:
            ridership_col = ridership_cols[0]
            top_stations = df_mta.groupby(station_col)[ridership_col].sum().sort_values(ascending=False).head(15)
            
            axes[0,0].barh(range(len(top_stations)), top_stations.values, color='navy')
            axes[0,0].set_yticks(range(len(top_stations)))
            axes[0,0].set_yticklabels([name[:25] + '...' if len(name) > 25 else name for name in top_stations.index], fontsize=8)
            axes[0,0].set_title('Top 15 Stations by Ridership\n(Economic activity hotspots)', fontweight='bold')
            axes[0,0].set_xlabel('Total Ridership')
            axes[0,0].grid(True, alpha=0.3)

# Borough distribution analysis
if any('borough' in col.lower() for col in df_mta.columns):
    borough_cols = [col for col in df_mta.columns if 'borough' in col.lower()]
    borough_col = borough_cols[0] if borough_cols else None
    
    if borough_col and ridership_cols:
        borough_ridership = df_mta.groupby(borough_col)[ridership_col].sum()
        
        axes[0,1].pie(borough_ridership.values, labels=borough_ridership.index, autopct='%1.1f%%', startangle=90)
        axes[0,1].set_title('Ridership Distribution by Borough\n(Economic activity concentration)', fontweight='bold')

# Hourly pattern analysis (if hour data available)
if hour_cols and ridership_cols:
    hour_col = hour_cols[0]
    hourly_pattern = df_mta.groupby(hour_col)[ridership_col].mean()
    
    axes[1,0].plot(hourly_pattern.index, hourly_pattern.values, marker='o', linewidth=2, color='darkred')
    axes[1,0].set_title('Average Hourly Ridership Pattern\n(Commuter vs. residential areas)', fontweight='bold')
    axes[1,0].set_xlabel('Hour of Day')
    axes[1,0].set_ylabel('Average Ridership')
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].set_xticks(range(0, 24, 2))

# Ridership distribution analysis
if ridership_cols:
    # Filter out extreme outliers for better visualization
    ridership_data = df_mta[ridership_col]
    q99 = ridership_data.quantile(0.99)
    filtered_ridership = ridership_data[ridership_data <= q99]
    
    axes[1,1].hist(filtered_ridership, bins=50, alpha=0.7, color='purple', edgecolor='black')
    axes[1,1].set_title('Ridership Volume Distribution\n(Economic activity intensity)', fontweight='bold')
    axes[1,1].set_xlabel('Ridership Count')
    axes[1,1].set_ylabel('Frequency')
    axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎯 KEY INSIGHT: MTA data reveals economic patterns invisible in other datasets:")
print("   • Station ridership volume indicates economic activity intensity")
print("   • Hourly patterns distinguish business districts from residential areas")
print("   • Geographic distribution shows economic clustering across boroughs")
print("   • Ridership trends reveal neighborhood vitality and accessibility")
print("\n💡 ONLY MTA DATA provides real-time economic activity proxies!")